In [1]:
import os
import numpy as np
import re
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers

2023-09-15 08:43:36.597930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 08:43:36.633285: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 08:43:36.633885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 08:43:37.234103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [3]:
MAX_WORDS_IN_SENTENCE = 700
WORD2VEC_DIMENSIONS = 50

In [4]:
embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [5]:
def text_to_word_vectors(text, vector_space_dimension):    
    sentence_vectors = []
    for word in text.lower().split():
        if word in embeddings_index and len(sentence_vectors) < MAX_WORDS_IN_SENTENCE:
            sentence_vectors.append(embeddings_index[word])    
    if len(sentence_vectors) > 0:
        pad_len = vector_space_dimension-len(sentence_vectors)      
        if pad_len > 0:
            padding = [np.zeros_like(sentence_vectors[0])] * pad_len
            sentence_vectors = sentence_vectors + padding
    return sentence_vectors

def clean_dataframe(df):
    cleaned_df = df.copy()
    pattern = r'[^a-zA-Z\s]'
    cleaned_df['text'] = cleaned_df['text'].astype(str).apply(lambda x: re.sub(pattern, '', x))    
    return cleaned_df

In [6]:
dataset = pd.read_csv('./data/recipe-dataset.csv')
dataset = clean_dataframe(dataset)
texts = dataset['text'].astype(str)
labels = dataset['label']

data = [text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE) for text in texts]
data = np.array(data)
targets = np.array(labels).astype("float32")

In [7]:
data.shape, targets.shape

((3328, 700, 50), (3328,))

In [8]:
test_x = data[:1500]
test_x = np.array(test_x).astype('float32')
test_x =  test_x.reshape(len(test_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
test_y = targets[:1500]

train_x = data[1500:]
train_x = np.array(train_x).astype('float32')
train_x = train_x.reshape(len(train_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
train_y = targets[1500:]

In [9]:
test_x.shape, test_y.shape

((1500, 35000), (1500,))

In [10]:
train_x.shape, train_y.shape

((1828, 35000), (1828,))

In [11]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS, )))  
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid")) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               3500100   
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 100)               5100      
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 3510351 (13.39 MB)
Trainable params: 35103

2023-09-15 08:44:18.244360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 08:44:18.245170: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_data=(test_x, test_y))

Epoch 1/5
58/58 [==============================] - 4s 52ms/step - loss: 0.0627 - accuracy: 0.9885 - val_loss: 0.0123 - val_accuracy: 0.9980
Epoch 2/5
58/58 [==============================] - 3s 48ms/step - loss: 0.0064 - accuracy: 0.9978 - val_loss: 9.7147e-04 - val_accuracy: 1.0000
Epoch 3/5
58/58 [==============================] - 3s 49ms/step - loss: 5.5591e-04 - accuracy: 1.0000 - val_loss: 6.6265e-04 - val_accuracy: 1.0000
Epoch 4/5
58/58 [==============================] - 3s 47ms/step - loss: 2.1645e-04 - accuracy: 1.0000 - val_loss: 6.3245e-04 - val_accuracy: 1.0000
Epoch 5/5
58/58 [==============================] - 3s 47ms/step - loss: 1.0243e-04 - accuracy: 1.0000 - val_loss: 8.2727e-04 - val_accuracy: 0.9993


In [13]:
model.save('./data/keras-w2v-recipe-50d.mdl')
loaded_model = models.load_model('./data/keras-w2v-recipe-50d.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe-50d.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe-50d.mdl/assets


Loaded Model Accuracy: 99.93%


In [14]:
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

Loaded Model Accuracy: 99.93%
